<a href="https://colab.research.google.com/github/Milechwan/if1015/blob/master/Projeto2_IF1015.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

dataset = pd.read_csv('https://raw.githubusercontent.com/Milechwan/if1015/master/googleplaystore.csv',encoding = "ISO-8859-1")

In [0]:
dataset.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite â FREE Live Cool Themes, Hid...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


Objetivo: Descobrir qual o melhor modelo para predizer a nota de avaliação (rating) dos 
aplicativos

In [2]:
dataset[dataset['Rating'].isna()]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
23,Mcqueen Coloring pages,ART_AND_DESIGN,NaN,61,7.0M,"100,000+",Free,0,Everyone,Art & Design;Action & Adventure,"March 7, 2018",1.0.0,4.1 and up
113,Wrinkles and rejuvenation,BEAUTY,NaN,182,5.7M,"100,000+",Free,0,Everyone 10+,Beauty,"September 20, 2017",8.0,3.0 and up
123,Manicure - nail design,BEAUTY,NaN,119,3.7M,"50,000+",Free,0,Everyone,Beauty,"July 23, 2018",1.3,4.1 and up
126,Skin Care and Natural Beauty,BEAUTY,NaN,654,7.4M,"100,000+",Free,0,Teen,Beauty,"July 17, 2018",1.15,4.1 and up
129,"Secrets of beauty, youth and health",BEAUTY,NaN,77,2.9M,"10,000+",Free,0,Mature 17+,Beauty,"August 8, 2017",2.0,2.3 and up
130,Recipes and tips for losing weight,BEAUTY,NaN,35,3.1M,"10,000+",Free,0,Everyone 10+,Beauty,"December 11, 2017",2.0,3.0 and up
134,"Lady adviser (beauty, health)",BEAUTY,NaN,30,9.9M,"10,000+",Free,0,Mature 17+,Beauty,"January 24, 2018",3.0,3.0 and up
163,Anonymous caller detection,BOOKS_AND_REFERENCE,NaN,161,2.7M,"10,000+",Free,0,Everyone,Books & Reference,"July 13, 2018",1.0,2.3 and up
180,SH-02J Owner's Manual (Android 8.0),BOOKS_AND_REFERENCE,NaN,2,7.2M,"50,000+",Free,0,Everyone,Books & Reference,"June 15, 2018",3.0,6.0 and up
185,URBANO V 02 instruction manual,BOOKS_AND_REFERENCE,NaN,114,7.3M,"100,000+",Free,0,Everyone,Books & Reference,"August 7, 2015",1.1,5.1 and up


Primeiramente, deve-se remover os dados faltantes e dados duplicados (comparando pelo nome)

In [3]:
dataset = dataset.dropna(subset=['Rating'])
dataset.drop_duplicates(subset=['App'],keep='last', inplace=True)
print(len(dataset))

8197


Transformação de colunas categóricas (definidas no Projeto 1), e remoção do símbolo $ da coluna Price para transformar em tipo numérico. Adicionalmente, é removida a linha (conforme foi também visto no Projeto 1) que possui os valores nas colunas incorretas, para transformar a coluna Reviews em tipo numérico.

In [0]:
dataset = dataset.drop(dataset[dataset['Price']=='Everyone'].index, axis = 0)
dataset['Reviews'] = pd.to_numeric(dataset['Reviews'])

In [0]:
dataset['Price'] = dataset['Price'].apply(lambda x: x[x.find('$')+1:] if x.find('$')>-1 else x)
dataset['Price'] = pd.to_numeric(dataset['Price'])
#dataset['Last Updated'] = pd.to_datetime(dataset['Last Updated'])

In [0]:
dataset['Type'] = dataset['Type'].astype("category")
dataset['Content Rating'] = dataset['Content Rating'].astype("category") 
dataset['Category'] = dataset['Category'].astype("category") 
dataset['Installs'] = dataset['Installs'].astype("category")
dataset['Android Ver'] = dataset['Android Ver'].astype("category")
dataset['Current Ver'] = dataset['Current Ver'].astype("category")
dataset['Size'] = dataset['Size'].astype("category")

dataset['Type'] = dataset['Type'].cat.codes
dataset['Content Rating'] = dataset['Content Rating'].cat.codes
dataset['Category'] = dataset['Category'].cat.codes
dataset['Installs'] = dataset['Installs'].cat.codes
dataset['Android Ver'] = dataset['Android Ver'].cat.codes
dataset['Current Ver'] = dataset['Current Ver'].cat.codes
dataset['Size'] = dataset['Size'].cat.codes

Remoção de outliers com DBSCAN

In [25]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

#Normalizando dataset para que o DBSCAN funcione corretamente para calcular as distâncias e clusterizar os dados
#removendo colunas inutilizadas 
#dataset = dataset.drop('Genres', axis = 1)
#dataset = dataset.drop('App', axis = 1)
#dataset = dataset.drop('Last Updated', axis = 1)
normalized_ds = scaler.fit_transform(dataset)
normalized_ds = pd.DataFrame(normalized_ds,columns=["Category","Rating","Reviews","Size","Installs","Type","Price","Content Rating","Current Ver","Android Ver"])
normalized_ds.head()

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Current Ver,Android Ver
0,0.0,0.775,0.000002,0.116223,0.277778,0.0,0.0,0.2,0.041143,0.483871
1,0.0,0.925,0.001120,0.808717,0.666667,0.0,0.0,0.2,0.161143,0.483871
2,0.0,0.875,0.002760,0.213075,0.833333,0.0,0.0,0.8,0.979048,0.580645
3,0.0,0.825,0.000012,0.138015,0.444444,0.0,0.0,0.2,0.095619,0.645161
4,0.0,0.850,0.000002,0.489104,0.777778,0.0,0.0,0.2,0.039619,0.258065


In [0]:
from sklearn.cluster import DBSCAN
outlier_detection = DBSCAN(
  eps = 0.5,
  metric="euclidean",
  min_samples = 10,
  n_jobs = -1)
clusters = outlier_detection.fit_predict(normalized_ds)

In [30]:
print(len(np.where(clusters == -1)))

1


Modelo de predição

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

model = LinearRegression()

X = np.array(dataset[['Reviews','Price','Installs','Android Ver','Current Ver','Category','Content Rating', 'Size']])
y = np.array(dataset[['Rating']])
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.33,
                                                random_state=42)

In [0]:
model.fit(Xtrain,ytrain)
y_model = model.predict(Xtest)
y_pred_train = model.predict(Xtrain)

In [9]:
print(model.score(Xtest,ytest))

0.007255682487009253


In [10]:
dataset.describe()

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Current Ver,Android Ver
count,8196.000000,8196.000000,8.196000e+03,8196.000000,8196.000000,8196.00000,8196.000000,8196.000000,8196.000000,8196.000000
mean,16.863104,4.173219,2.553206e+05,185.811127,7.489507,0.07345,1.035447,1.458272,1195.501952,16.033553
std,8.313741,0.536269,1.984960e+06,137.697398,4.880289,0.26089,16.857244,1.002664,861.440704,6.793599
min,0.000000,1.000000,1.000000e+00,0.000000,0.000000,0.00000,0.000000,0.000000,-1.000000,-1.000000
25%,11.000000,4.000000,1.267500e+02,64.000000,2.000000,0.00000,0.000000,1.000000,394.000000,12.000000
50%,14.000000,4.300000,3.009500e+03,151.000000,6.000000,0.00000,0.000000,1.000000,1034.000000,16.000000
75%,25.000000,4.500000,4.371950e+04,292.000000,12.000000,0.00000,0.000000,1.000000,1948.000000,19.000000
max,32.000000,5.000000,7.812821e+07,413.000000,18.000000,1.00000,400.000000,5.000000,2624.000000,30.000000


Usando SelectFromModel, que dado o estimador e as variáveis X e y, ele retorna um array contendo toda a base mas selecionando as variáveis independentes mais adequadas para fazer a predição

In [14]:
from sklearn.feature_selection import SelectFromModel
feature = SelectFromModel(model)
Fit = feature.fit_transform(X, y)


{'estimator__copy_X': True, 'estimator__fit_intercept': True, 'estimator__n_jobs': None, 'estimator__normalize': False, 'estimator': LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False), 'max_features': None, 'norm_order': 1, 'prefit': False, 'threshold': None}


In [17]:
print(Fit.shape)

(8196, 3)


In [18]:
Fittrain, Fittest, ytrain, ytest = train_test_split(Fit, y, test_size=0.33,
                                                random_state=42)
model.fit(Fittrain,ytrain)
y_model = model.predict(Fittest)
print(model.score(Fittest,ytest))

0.0022569904097181626


RandomForest

In [0]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 100, random_state = 42)

In [0]:
rf.fit(Xtrain,ytrain)

model = rf.predict(Xtest)

errors = abs(model - ytest)
print('Mean Absolute Error:', round(np.mean(errors), 2))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


Mean Absolute Error: 0.43


In [0]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / ytest)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 88.36 %.
